# **Inicializar Ambiente**

In [1]:
import pandas as pd
import sqlite3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

database = '/content/drive/MyDrive/Colab Notebooks/Talento Tech/Módulo 2/Manos A La Obra/database.sqlite'
conn = sqlite3.connect(database)

Mounted at /content/drive


In [3]:
def consultar(query):
    return pd.read_sql(query, conn)

# **1. ¿Cuántos Partidos Se Han Jugado En Total Por Cada Liga?**

In [4]:
consultar("""
    SELECT
	    COUNT(*) AS Partidos,
	    L.name AS Liga
    FROM
    	Match M
    	JOIN
		    League L
		    ON L.id = M.league_id
    GROUP BY
    	Liga
    ORDER BY
    	Partidos ASC;
""")

,Partidos,Liga
0,1422,Switzerland Super League
1,1728,Belgium Jupiler League
2,1824,Scotland Premier League
3,1920,Poland Ekstraklasa
4,2052,Portugal Liga ZON Sagres
5,2448,Germany 1. Bundesliga
6,2448,Netherlands Eredivisie
7,3017,Italy Serie A
8,3040,England Premier League
9,3040,France Ligue 1


# **2. ¿Cuál Es El Porcentaje De Victorias, Derrotas Y Empates De Los Equipos Locales?**

In [5]:
consultar("""
    SELECT
	    Resultado,
        ROUND((COUNT(Resultado) * 100.0 / (SELECT COUNT(*) FROM Match)), 2) AS Porcentaje
    FROM (
        SELECT
            CASE
                WHEN home_team_goal > away_team_goal THEN 'Victoria'
                WHEN home_team_goal = away_team_goal THEN 'Empate'
                ELSE 'Derrota'
            END AS Resultado
        FROM
            Match
    )
    GROUP BY
        Resultado
    ORDER BY
        Porcentaje DESC;
""")

,Resultado,Porcentaje
0,Victoria,45.87
1,Derrota,28.74
2,Empate,25.39


# **3. ¿Qué Equipos Tienen La Mayor Cantidad De Victorias Como Visitantes?**

In [6]:
consultar("""
    SELECT
    	COUNT(*) AS Victorias,
        T.team_long_name AS Equipo
    FROM
	    Match M
	    JOIN
		    Team T
		    ON T.team_api_id = M.away_team_api_id
    WHERE
	    M.away_team_goal > M.home_team_goal
    GROUP BY
	    T.team_long_name
    ORDER BY
	    Victorias DESC
	LIMIT
		10;
""")

,Victorias,Equipo
0,103,FC Barcelona
1,99,Real Madrid CF
2,98,Celtic
3,84,Juventus
4,84,FC Bayern Munich
5,83,SL Benfica
6,81,FC Porto
7,78,Ajax
8,77,FC Basel
9,76,Manchester United


# **4. ¿Cuál Es El Porcentaje De Victorias, Derrotas Y Empates De Los Equipos Visitantes En La Liga BBVA?**

In [15]:
consultar("""
    SELECT
	    Resultado,
        ROUND((COUNT(Resultado) * 100.0 / (SELECT COUNT(*) FROM Match WHERE league_id = 21518)), 2) AS Porcentaje
    FROM (
        SELECT
            CASE
                WHEN home_team_goal < away_team_goal THEN 'Victoria'
                WHEN home_team_goal = away_team_goal THEN 'Empate'
                ELSE 'Derrota'
            END AS Resultado
        FROM
            Match
        WHERE
        	league_id = 21518
    )
    GROUP BY
        Resultado
    ORDER BY
        Porcentaje DESC;
""")

,Resultado,Porcentaje
0,Derrota,48.85
1,Victoria,27.99
2,Empate,23.16


# **5. ¿Cuáles Han Sido Los Equipos Con Más Goles En Total?**

In [8]:
consultar("""
    SELECT
        TH.team_long_name AS Equipo,
        L.name AS Liga,
        SUM(CASE WHEN M.season <= '2008/2009' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2008/2009',
        SUM(CASE WHEN M.season <= '2009/2010' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2009/2010',
        SUM(CASE WHEN M.season <= '2010/2011' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2010/2011',
        SUM(CASE WHEN M.season <= '2011/2012' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2011/2012',
        SUM(CASE WHEN M.season <= '2012/2013' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2012/2013',
        SUM(CASE WHEN M.season <= '2013/2014' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2013/2014',
        SUM(CASE WHEN M.season <= '2014/2015' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2014/2015',
        SUM(CASE WHEN M.season <= '2015/2016' THEN M.home_team_goal + M.away_team_goal ELSE 0 END) AS '2015/2016'
    FROM
	    Match M
        JOIN
            Team TH
            ON M.home_team_api_id = TH.team_api_id
        JOIN
            Team TA
            ON M.away_team_api_id = TA.team_api_id
        JOIN
            League L
            ON M.league_id = L.id
    GROUP BY
        TH.team_long_name
    ORDER BY
    	Equipo;
""")

,Equipo,Liga,2008/2009,2009/2010,2010/2011,2011/2012,2012/2013,2013/2014,2014/2015,2015/2016
0,1. FC Kaiserslautern,Germany 1. Bundesliga,0,0,44,84,84,84,84,84
1,1. FC Köln,Germany 1. Bundesliga,39,86,137,186,186,186,221,255
2,1. FC Nürnberg,Germany 1. Bundesliga,0,41,88,135,178,226,226,226
3,1. FSV Mainz 05,Germany 1. Bundesliga,0,36,74,127,168,213,259,300
4,AC Ajaccio,France Ligue 1,0,0,0,46,85,141,141,141
...,...,...,...,...,...,...,...,...,...,...
291,Xerez Club Deportivo,Spain LIGA BBVA,0,49,49,49,49,49,49,49
292,Zagłębie Lubin,Poland Ekstraklasa,0,30,56,94,136,180,180,224
293,Zawisza Bydgoszcz,Poland Ekstraklasa,0,0,0,0,0,43,81,81
294,Évian Thonon Gaillard FC,France Ligue 1,0,0,0,58,114,163,208,208


# **6. ¿Cómo Varían Las Estadísticas De Messi Contra Las De Cristiano Ronaldo A Lo Largo De La Historia?**

In [9]:
consultar("""
    SELECT
        SUBSTR(M.date, 0, 11) AS Fecha,
        'Messi' AS LM10,
        M.finishing AS Definición,
        M.heading_accuracy AS 'Precisión De Cabeceo',
        M.dribbling AS Regate,
        M.free_kick_accuracy AS 'Precisión De Tiros Libres',
        M.long_passing AS 'Pases Largos',
        M.ball_control AS 'Control Del Balón',
        M.shot_power AS 'Potencia De Tiro',
        M.long_shots AS 'Tiros Lejanos',
        M.vision AS Visión,
        M.penalties AS Penaltis,
        'Cristiano Ronaldo' AS CR7,
        R.finishing AS Definición,
        R.heading_accuracy AS 'Precisión De Cabeceo',
        R.dribbling AS Regate,
        R.free_kick_accuracy AS 'Precisión De Tiros Libres',
        R.long_passing AS 'Pases Largos',
        R.ball_control AS 'Control Del Balón',
        R.shot_power AS 'Potencia De Tiro',
        R.long_shots AS 'Tiros Lejanos',
        R.vision AS Visión,
        R.penalties AS Penaltis
    FROM
        Player P_Messi
        JOIN
            Player_Attributes M
            ON P_Messi.player_api_id = M.player_api_id
        JOIN
            Player P_Ronaldo
            ON P_Ronaldo.player_name = 'Cristiano Ronaldo'
        JOIN
            Player_Attributes R
            ON P_Ronaldo.player_api_id = R.player_api_id
    WHERE
        P_Messi.player_name = 'Lionel Messi'
        AND M.date = R.date
    ORDER BY
        M.date;
""")

,Fecha,LM10,Definición,Precisión De Cabeceo,Regate,Precisión De Tiros Libres,Pases Largos,Control Del Balón,Potencia De Tiro,Tiros Lejanos,...,Definición,Precisión De Cabeceo,Regate,Precisión De Tiros Libres,Pases Largos,Control Del Balón,Potencia De Tiro,Tiros Lejanos,Visión,Penaltis
0,2007-02-22,Messi,76,59,95,87,79,92,83,84,...,71,74,95,64,68,92,80,72,80,64
1,2007-08-30,Messi,80,63,95,70,79,94,84,84,...,85,84,97,88,73,95,90,75,80,79
2,2008-08-30,Messi,87,64,97,70,79,95,82,82,...,92,82,97,94,72,96,92,85,80,78
3,2009-02-22,Messi,87,64,97,76,79,95,79,82,...,91,82,97,93,71,95,90,84,80,78
4,2009-08-30,Messi,90,64,97,74,75,96,79,84,...,90,88,93,88,71,95,91,87,80,86
5,2010-02-22,Messi,90,66,97,74,75,96,79,84,...,90,87,93,88,71,95,92,89,80,86
6,2010-08-30,Messi,91,69,97,82,75,96,79,87,...,89,87,93,88,71,95,93,91,80,82
7,2011-02-22,Messi,92,69,96,82,77,96,79,89,...,90,87,93,87,72,95,93,90,81,84
8,2011-08-30,Messi,92,69,97,82,75,97,81,88,...,90,87,93,87,72,95,93,90,81,87
9,2012-02-22,Messi,93,73,97,82,75,97,80,88,...,90,87,93,87,72,95,93,90,81,91


# **7. ¿Cuál Fue El Promedio De Goles Por Liga Por Cada Año?**

In [10]:
consultar("""
    SELECT
        L.name AS Liga,
        ROUND(AVG(CASE WHEN M.season = '2008/2009' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2008/2009',
        ROUND(AVG(CASE WHEN M.season = '2009/2010' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2009/2010',
        ROUND(AVG(CASE WHEN M.season = '2010/2011' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2010/2011',
        ROUND(AVG(CASE WHEN M.season = '2011/2012' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2011/2012',
        ROUND(AVG(CASE WHEN M.season = '2012/2013' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2012/2013',
        ROUND(AVG(CASE WHEN M.season = '2013/2014' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2013/2014',
        ROUND(AVG(CASE WHEN M.season = '2014/2015' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2014/2015',
        ROUND(AVG(CASE WHEN M.season = '2015/2016' THEN M.home_team_goal + M.away_team_goal END), 3) AS '2015/2016'
    FROM
    	Match M
        JOIN
            League L
            ON M.league_id = L.id
    GROUP BY
        Liga
    ORDER BY
    	Liga;
""")

,Liga,2008/2009,2009/2010,2010/2011,2011/2012,2012/2013,2013/2014,2014/2015,2015/2016
0,Belgium Jupiler League,2.794,2.690,2.646,2.879,2.929,2.500,2.783,2.892
1,England Premier League,2.479,2.771,2.797,2.805,2.797,2.768,2.566,2.700
2,France Ligue 1,2.258,2.411,2.342,2.516,2.545,2.455,2.492,2.526
3,Germany 1. Bundesliga,2.922,2.830,2.922,2.859,2.935,3.160,2.755,2.830
4,Italy Serie A,2.600,2.611,2.513,2.584,2.639,2.724,2.686,2.576
5,Netherlands Eredivisie,2.843,2.915,3.225,3.258,3.150,3.196,3.078,2.980
6,Poland Ekstraklasa,2.183,2.217,2.408,2.196,2.492,2.642,2.617,2.646
7,Portugal Liga ZON Sagres,2.300,2.504,2.433,2.642,2.779,2.371,2.493,2.716
8,Scotland Premier League,2.404,2.566,2.561,2.636,2.732,2.746,2.575,2.851
9,Spain LIGA BBVA,2.897,2.713,2.742,2.763,2.871,2.750,2.655,2.745


# **8. ¿Qué Correlación Existe Entre El Índice De Masa Corporal Y La Velocidad De Los Jugadores?**

In [11]:
consultar("""
    WITH Valores_Adecuados AS (
	    SELECT
    		player_api_id AS id,
		    player_name AS 'Jugador',
		    ROUND(weight/2.205, 2) AS Peso,
		    ROUND(height/100, 2) AS Altura
	    FROM
    		Player
    )

    SELECT
    	VA.Jugador,
    	ROUND(VA.Peso / (VA.Altura * VA.Altura), 2) AS IMC,
    	PA.sprint_speed AS Velocidad,
    	PA.agility AS Agilidad
    FROM
    	Valores_Adecuados VA
    	JOIN
		    Player_Attributes PA
		    ON VA.id = PA.player_api_id
    GROUP BY
    	VA.Jugador;
""")

,Jugador,IMC,Velocidad,Agilidad
0,Aaron Appindangoye,25.32,64,59.0
1,Aaron Cresswell,22.91,78,78.0
2,Aaron Doran,25.58,74,85.0
3,Aaron Galindo,26.81,34,62.0
4,Aaron Hughes,20.85,31,41.0
...,...,...,...,...
10843,Zsolt Low,21.56,73,59.0
10844,Zurab Khizanishvili,22.79,66,56.0
10845,Zvjezdan Misimovic,24.64,63,81.0
10846,de Oliveira Cleber Monteiro,23.02,59,57.0


# **9. ¿Qué Correlación Hay Entre La Creación De Oportunidades Mediante Pases Y El Total De Goles Por Liga?**

In [12]:
consultar("""
    WITH Equipos AS (
    	SELECT
		    TA.chanceCreationPassing,
		    T.team_long_name AS Equipo,
		    T.team_api_id AS id
	    FROM
    		Team_Attributes TA
    	JOIN
		    TEAM T
		    ON TA.team_api_id = T.team_api_id
	    GROUP BY
    		Equipo
    )

    SELECT
    	E.Equipo,
    	L.name AS Liga,
    	SUM(M.home_team_goal + M.away_team_goal) AS Goles,
    	E.chanceCreationPassing AS 'Oportunidades Por Pases'
    FROM
    	Equipos E
    	JOIN
		    Match M
		    ON M.home_team_api_id = E.id
	    JOIN
    		League L
		    ON L.id = M.league_id
    GROUP BY
    	E.Equipo;
""")

,Equipo,Liga,Goles,Oportunidades Por Pases
0,1. FC Kaiserslautern,Germany 1. Bundesliga,84,45
1,1. FC Köln,Germany 1. Bundesliga,255,65
2,1. FC Nürnberg,Germany 1. Bundesliga,226,40
3,1. FSV Mainz 05,Germany 1. Bundesliga,300,45
4,AC Ajaccio,France Ligue 1,141,45
...,...,...,...,...
280,Xerez Club Deportivo,Spain LIGA BBVA,49,45
281,Zagłębie Lubin,Poland Ekstraklasa,224,30
282,Zawisza Bydgoszcz,Poland Ekstraklasa,81,67
283,Évian Thonon Gaillard FC,France Ligue 1,208,50


# **10. ¿Cómo Es La Distribución De Los Resultados En Cada Una De Las Casas De Apuestas Y Cuál Es El "Overround" En Cada Una De Estas?**

In [14]:
consultar("""
    WITH Probabilidades AS (
        SELECT
            AVG(1.0 / B365H) AS B365H_PI, AVG(1.0 / B365D) AS B365D_PI, AVG(1.0 / B365A) AS B365A_PI,

            AVG(1.0 / BWH) AS BWH_PI, AVG(1.0 / BWD) AS BWD_PI, AVG(1.0 / BWA) AS BWA_PI,

            AVG(1.0 / IWH) AS IWH_PI, AVG(1.0 / IWD) AS IWD_PI, AVG(1.0 / IWA) AS IWA_PI,

            AVG(1.0 / LBH) AS LBH_PI, AVG(1.0 / LBD) AS LBD_PI, AVG(1.0 / LBA) AS LBA_PI,

            AVG(1.0 / PSH) AS PSH_PI, AVG(1.0 / PSD) AS PSD_PI, AVG(1.0 / PSA) AS PSA_PI,

            AVG(1.0 / WHH) AS WHH_PI, AVG(1.0 / WHD) AS WHD_PI, AVG(1.0 / WHA) AS WHA_PI,

            AVG(1.0 / SJH) AS SJH_PI, AVG(1.0 / SJD) AS SJD_PI, AVG(1.0 / SJA) AS SJA_PI,

            AVG(1.0 / VCH) AS VCH_PI, AVG(1.0 / VCD) AS VCD_PI, AVG(1.0 / VCA) AS VCA_PI,

            AVG(1.0 / GBH) AS GBH_PI, AVG(1.0 / GBD) AS GBD_PI, AVG(1.0 / GBA) AS GBA_PI,

            AVG(1.0 / BSH) AS BSH_PI, AVG(1.0 / BSD) AS BSD_PI, AVG(1.0 / BSA) AS BSA_PI
        FROM Match
    ),

    Resultados AS (
        SELECT
            *,

            (B365H_PI + B365D_PI + B365A_PI) AS Total_B365,
            ROUND(((B365H_PI + B365D_PI + B365A_PI) - 1) * 100, 2) || '%' AS B365_Overround,

            (BWH_PI + BWD_PI + BWA_PI) AS Total_BWH,
            ROUND(((BWH_PI + BWD_PI + BWA_PI) - 1) * 100, 2) || '%' AS BWH_Overround,

            (IWH_PI + IWD_PI + IWA_PI) AS Total_IWH,
            ROUND(((IWH_PI + IWD_PI + IWA_PI) - 1) * 100, 2) || '%' AS IWH_Overround,

            (LBH_PI + LBD_PI + LBA_PI) AS Total_LBH,
            ROUND(((LBH_PI + LBD_PI + LBA_PI) - 1) * 100, 2) || '%' AS LBH_Overround,

            (PSH_PI + PSD_PI + PSA_PI) AS Total_PSH,
            ROUND(((PSH_PI + PSD_PI + PSA_PI) - 1) * 100, 2) || '%' AS PSH_Overround,

            (WHH_PI + WHD_PI + WHA_PI) AS Total_WHH,
            ROUND(((WHH_PI + WHD_PI + WHA_PI) - 1) * 100, 2) || '%' AS WHH_Overround,

            (SJH_PI + SJD_PI + SJA_PI) AS Total_SJH,
            ROUND(((SJH_PI + SJD_PI + SJA_PI) - 1) * 100, 2) || '%' AS SJH_Overround,

            (VCH_PI + VCD_PI + VCA_PI) AS Total_VCH,
            ROUND(((VCH_PI + VCD_PI + VCA_PI) - 1) * 100, 2) || '%' AS VCH_Overround,

            (GBH_PI + GBD_PI + GBA_PI) AS Total_GBH,
            ROUND(((GBH_PI + GBD_PI + GBA_PI) - 1) * 100, 2) || '%' AS GBH_Overround,

            (BSH_PI + BSD_PI + BSA_PI) AS Total_BSH,
            ROUND(((BSH_PI + BSD_PI + BSA_PI) - 1) * 100, 2) || '%' AS BSH_Overround
        FROM Probabilidades
    )

    SELECT 'Bet365' AS Casa, 'Victoria Local' AS Resultado, ROUND((B365H_PI / Total_B365) * 100, 2) AS Valor, B365_Overround AS Overround
    FROM Resultados UNION ALL
    SELECT 'Bet365', 'Empate', ROUND((B365D_PI / Total_B365) * 100, 2), B365_Overround
    FROM Resultados UNION ALL
    SELECT 'Bet365', 'Victoria Visitante', ROUND((B365A_PI / Total_B365) * 100, 2), B365_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Betfair', 'Victoria Local', ROUND((BWH_PI / Total_BWH) * 100, 2), BWH_Overround
    FROM Resultados UNION ALL
    SELECT 'Betfair', 'Empate', ROUND((BWD_PI / Total_BWH) * 100, 2), BWH_Overround
    FROM Resultados UNION ALL
    SELECT 'Betfair', 'Victoria Visitante', ROUND((BWA_PI / Total_BWH) * 100, 2), BWH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Interwetten', 'Victoria Local', ROUND((IWH_PI / Total_IWH) * 100, 2), IWH_Overround
    FROM Resultados UNION ALL
    SELECT 'Interwetten', 'Empate', ROUND((IWD_PI / Total_IWH) * 100, 2), IWH_Overround
    FROM Resultados UNION ALL
    SELECT 'Interwetten', 'Victoria Visitante', ROUND((IWA_PI / Total_IWH) * 100, 2), IWH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Ladbrokes', 'Victoria Local', ROUND((LBH_PI / Total_LBH) * 100, 2), LBH_Overround
    FROM Resultados UNION ALL
    SELECT 'Ladbrokes', 'Empate', ROUND((LBD_PI / Total_LBH) * 100, 2), LBH_Overround
    FROM Resultados UNION ALL
    SELECT 'Ladbrokes', 'Victoria Visitante', ROUND((LBA_PI / Total_LBH) * 100, 2), LBH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Pinnacle Sports', 'Victoria Local', ROUND((PSH_PI / Total_PSH) * 100, 2), PSH_Overround
    FROM Resultados UNION ALL
    SELECT 'Pinnacle Sports', 'Empate', ROUND((PSD_PI / Total_PSH) * 100, 2), PSH_Overround
    FROM Resultados UNION ALL
    SELECT 'Pinnacle Sports', 'Victoria Visitante', ROUND((PSA_PI / Total_PSH) * 100, 2), PSH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'William Hill', 'Victoria Local', ROUND((WHH_PI / Total_WHH) * 100, 2), WHH_Overround
    FROM Resultados UNION ALL
    SELECT 'William Hill', 'Empate', ROUND((WHD_PI / Total_WHH) * 100, 2), WHH_Overround
    FROM Resultados UNION ALL
    SELECT 'William Hill', 'Victoria Visitante', ROUND((WHA_PI / Total_WHH) * 100, 2), WHH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'SBOBET', 'Victoria Local', ROUND((SJH_PI / Total_SJH) * 100, 2), SJH_Overround
    FROM Resultados UNION ALL
    SELECT 'SBOBET', 'Empate', ROUND((SJD_PI / Total_SJH) * 100, 2), SJH_Overround
    FROM Resultados UNION ALL
    SELECT 'SBOBET', 'Victoria Visitante', ROUND((SJA_PI / Total_SJH) * 100, 2), SJH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Victor Chandler', 'Victoria Local', ROUND((VCH_PI / Total_VCH) * 100, 2), VCH_Overround
    FROM Resultados UNION ALL
    SELECT 'Victor Chandler', 'Empate', ROUND((VCD_PI / Total_VCH) * 100, 2), VCH_Overround
    FROM Resultados UNION ALL
    SELECT 'Victor Chandler', 'Victoria Visitante', ROUND((VCA_PI / Total_VCH) * 100, 2), VCH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Gamebookers', 'Victoria Local', ROUND((GBH_PI / Total_GBH) * 100, 2), GBH_Overround
    FROM Resultados UNION ALL
    SELECT 'Gamebookers', 'Empate', ROUND((GBD_PI / Total_GBH) * 100, 2), GBH_Overround
    FROM Resultados UNION ALL
    SELECT 'Gamebookers', 'Victoria Visitante', ROUND((GBA_PI / Total_GBH) * 100, 2), GBH_Overround
    FROM Resultados

    UNION ALL

    SELECT 'Betsson', 'Victoria Local', ROUND((BSH_PI / Total_BSH) * 100, 2), BSH_Overround
    FROM Resultados UNION ALL
    SELECT 'Betsson', 'Empate', ROUND((BSD_PI / Total_BSH) * 100, 2), BSH_Overround
    FROM Resultados UNION ALL
    SELECT 'Betsson', 'Victoria Visitante', ROUND((BSA_PI / Total_BSH) * 100, 2), BSH_Overround
    FROM Resultados;
""")

,Casa,Resultado,Valor,Overround
0,Bet365,Victoria Local,45.06,6.15%
1,Bet365,Empate,25.78,6.15%
2,Bet365,Victoria Visitante,29.16,6.15%
3,Betfair,Victoria Local,44.75,8.05%
4,Betfair,Empate,25.85,8.05%
5,Betfair,Victoria Visitante,29.40,8.05%
6,Interwetten,Victoria Local,44.46,9.92%
7,Interwetten,Empate,26.05,9.92%
8,Interwetten,Victoria Visitante,29.49,9.92%
9,Ladbrokes,Victoria Local,44.76,8.68%
